<a href="https://colab.research.google.com/github/lachchi/saritha/blob/main/meetmate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask transformers openai-whisper googletrans==4.0.0-rc1
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!apt install ffmpeg -y


Looking in indexes: https://download.pytorch.org/whl/cpu
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import os
import threading
from flask import Flask, request, render_template_string
import whisper
from transformers import pipeline
from googletrans import Translator
from google.colab import output

# Load models
whisper_model = whisper.load_model("base")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
translator = Translator()

app = Flask(__name__)
stored_transcript = ""
stored_summary = ""

# Styled HTML Template
HTML = """
<!doctype html>
<html>
<head>
  <title>MeetMate: Smart Meeting Assistant</title>
  <style>
    body {
      font-family: 'Segoe UI', sans-serif;
      background: #f4f7f9;
      color: #333;
      padding: 30px;
      max-width: 900px;
      margin: auto;
    }
    h2, h3 {
      color: #2c3e50;
    }
    form, .output-box {
      background: #fff;
      padding: 20px;
      border-radius: 8px;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
      margin-bottom: 30px;
    }
    input[type="file"], input[type="text"], select {
      padding: 10px;
      margin-top: 10px;
      width: 100%;
      box-sizing: border-box;
    }
    input[type="submit"] {
      background-color: #3498db;
      color: white;
      padding: 10px 15px;
      border: none;
      border-radius: 4px;
      margin-top: 10px;
      cursor: pointer;
    }
    input[type="submit"]:hover {
      background-color: #2980b9;
    }
    pre {
      white-space: pre-wrap;
      word-wrap: break-word;
    }
  </style>
</head>
<body>
  <h2>📋 MeetMate: Smart Meeting Assistant</h2>

  <form method=post enctype=multipart/form-data>
    <label>🎧 Upload a Meeting Audio File (.mp3/.wav):</label><br>
    <input type="file" name="audio" accept="audio/*" required><br>
    <input type="submit" value="Transcribe & Summarize">
  </form>

  {% if transcript %}
  <div class="output-box">
    <h3>📝 Transcript:</h3>
    <pre>{{ transcript }}</pre>

    <h3>📌 Summary:</h3>
    <p>{{ summary }}</p>
  </div>

  <form method="post">
    <h3>🔍 Ask a Question:</h3>
    <input type="text" name="question" placeholder="What was discussed about the budget?" required>
    <input type="submit" value="Get Answer">
  </form>

  {% if answer %}
    <div class="output-box">
      <h3>🤖 Answer:</h3>
      <p>{{ answer }}</p>
    </div>
  {% endif %}

  <form method="post">
    <h3>🌐 Translate Summary:</h3>
    <select name="language">
      <option value="hi">Hindi</option>
      <option value="ta">Tamil</option>
      <option value="fr">French</option>
      <option value="es">Spanish</option>
    </select>
    <input type="submit" value="Translate">
  </form>

  {% if translated %}
    <div class="output-box">
      <h3>🌎 Translated Summary:</h3>
      <p>{{ translated }}</p>
    </div>
  {% endif %}
  {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def index():
    global stored_transcript, stored_summary
    transcript = summary = answer = translated = None

    # Audio upload
    if 'audio' in request.files:
        audio = request.files['audio']
        ext = audio.filename.rsplit('.', 1)[-1]
        audio_path = f"temp_audio.{ext}"
        audio.save(audio_path)

        result = whisper_model.transcribe(audio_path)
        transcript = result["text"]
        stored_transcript = transcript

        summary_output = summarizer(transcript, max_length=150, min_length=40, do_sample=False)
        summary = summary_output[0]['summary_text']
        stored_summary = summary

        os.remove(audio_path)

    # Q&A
    if 'question' in request.form:
        question = request.form['question']
        if stored_transcript:
            try:
                result = qa_pipeline(question=question, context=stored_transcript)
                answer = result['answer']
            except:
                answer = "Sorry, I couldn't find an answer."

    # Translation
    if 'language' in request.form:
        lang = request.form['language']
        if stored_summary:
            try:
                translated = translator.translate(stored_summary, dest=lang).text
            except:
                translated = "Translation failed."

    return render_template_string(HTML,
                                  transcript=stored_transcript if stored_transcript else None,
                                  summary=stored_summary if stored_summary else None,
                                  answer=answer,
                                  translated=translated)

# Start Flask in background
def run_flask():
    app.run(host='0.0.0.0', port=5000)

threading.Thread(target=run_flask).start()
output.serve_kernel_port_as_window(5000)


Device set to use cpu
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


 * Serving Flask app '__main__'
Try `serve_kernel_port_as_iframe` instead. 
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


<IPython.core.display.Javascript object>

 * Debug mode: off
